In [4]:
#source1: https://nbviewer.org/github/ekontou/CEE-UTM-UTO/tree/master/FrankWolfeforUE-Lecture6.ipynb
#source2: https://nbviewer.org/github/ekontou/CEE-UTM-UTO/tree/master/NetworkX-Basics-v1-EK.ipynb
#source3: https://datatofish.com/read_excel/

import pandas as pd
import numpy as np
import networkx as nx
import scipy.optimize as sopt

#import link and demand data
df1 = pd.read_excel ('TestData.xlsx', sheet_name='LinkCharacteristics')
df2 = pd.read_excel ('TestData.xlsx', sheet_name='O-D matrix')
linkfun = df1.to_numpy()
demand = df2.to_numpy()

#define link performance function
def t(x, array):
    i=0
    t=[]
    while (i<len(linkfun)):
        t.append(linkfun[i,3]+linkfun[i,4]*x[i])
        i+=1
    return t

#define objective function (sum of integral of t)
def z(x, array):
    i=0
    z=0
    while (i<len(linkfun)):
        z+=linkfun[i,3]*x[i]+1/2*linkfun[i,4]*(x[i])**2
        i+=1
    return z

#implement Dijkstra's Algorithm
def sp(x, linkfun, t, O, D):
    
    #initialize an empty directed graph object
    G1=nx.DiGraph()

    #add nodes to graph
    i=0
    while (i<len(linkfun)):
        if (i==0):
            G1.add_node(linkfun[i,1])
        elif(linkfun[i,0]!=linkfun[i-1,1]):
            G1.add_node(linkfun[i,1])
        i+=1
 
    #add link characteristics
    ti = t(x,linkfun)
    i=0
    while (i<len(linkfun)):
        G1.add_edge(linkfun[i,1],linkfun[i,2], weight = ti[i])
        i+=1
        
    #find shortest path
    sp = nx.dijkstra_path(G1,source = O, target = D)
    return sp

#implement Frank-Wolfe Algorithm
#at Iteration 0
guesses = np.zeros(len(linkfun))
i=0
while (i<len(demand)):
    path = sp(guesses, linkfun, t, demand[i,0], demand[i,1])
    j=0
    while (j<len(linkfun)):
        k=0
        while (k<len(path)-1):
            if(linkfun[j,1]==path[k] and linkfun[j,2]==path[k+1]):
                guesses[j]+=demand[i,2]
            k+=1
        j+=1
    i+=1

#at subsequent iterations
def step(s, guesses):
    
    #find direction y based on min t
    y = np.zeros(len(linkfun))
    i=0
    while (i<len(demand)):
        path = sp(guesses, linkfun, t, demand[i,0], demand[i,1])
        j=0
        while (j<len(linkfun)):
            k=0
            while (k<len(path)-1):
                if(linkfun[j,1]==path[k] and linkfun[j,2]==path[k+1]):
                    y[j]+=demand[i,2]
                k+=1
            j+=1
        i+=1

    # objective function to minimize
    def func(alpha):
        return z(guesses+alpha*(y-guesses),linkfun)

    # find alpha that minimizes objective function
    alpha = sopt.minimize(func, 0, bounds=[(0,1)])

    # update x
    last_guess = guesses
    guesses = guesses + alpha.x*(y-guesses)
    return last_guess,guesses

def converge_test(last_guess,next_guess):
    delta = 0.00001
    v1=v2=check=i=0
    while (i<len(guesses)):
        v1 += (next_guess[i]-last_guess[i])**2
        v2 += next_guess[i]
        i+=1
    check=v1**0.5/v2
    return True if (check < delta) else False
          
for s in range(1,100):
    last_guess,next_guess=step(s,guesses)
    guesses=next_guess
    if converge_test(last_guess,next_guess)==True:
        break

print('x =',np.round(guesses,0))
print('t =',np.round(t(guesses,linkfun),3))
print('T =',np.round(np.sum(t(guesses,linkfun)*guesses),0))

x = [ 100.   75. 1900.  925.  175.]
t = [0.6   0.575 1.95  1.925 1.35 ]
T = 5825.0
